In [1]:
!pip install openai

  Obtaining dependency information for openai from https://files.pythonhosted.org/packages/69/95/22a9a81cebd54e18841da429f05f06ed867648768f7af938ad34f13197fd/openai-1.3.3-py3-none-any.whl.metadata
  Using cached openai-1.3.3-py3-none-any.whl.metadata (16 kB)
  Using cached distro-1.8.0-py3-none-any.whl (20 kB)
  Obtaining dependency information for httpx<1,>=0.23.0 from https://files.pythonhosted.org/packages/82/61/a5fca4a1e88e40969bbd0cf0d981f3aa76d5057db160b94f49603fc18740/httpx-0.25.1-py3-none-any.whl.metadata
  Using cached httpx-0.25.1-py3-none-any.whl.metadata (7.1 kB)
  Obtaining dependency information for pydantic<3,>=1.9.0 from https://files.pythonhosted.org/packages/e2/2c/9906b7abc337b0250a5634de5396e2f3cb1b837af0616424c2225a65aa80/pydantic-2.5.1-py3-none-any.whl.metadata
  Using cached pydantic-2.5.1-py3-none-any.whl.metadata (64 kB)
  Obtaining dependency information for httpcore from https://files.pythonhosted.org/packages/56/ba/78b0a99c4da0ff8b0f59defa2f13ca4668189b134bd9

In [2]:
import openai
import os
# Check to see if there is an environment variable with you API keys, if not, use what you put below
openai.api_key = os.environ.get('OPENAI_API_KEY',"")

In [28]:
from openai import OpenAI
client = OpenAI(
  api_key=os.environ.get('OPENAI_API_KEY',""), 
)

response = client.chat.completions.create(
  model="gpt-4-1106-preview",
  messages=[
    {
      "role": "system",
      "content": "You will generate a Python Script that will complete the task outlined by the user. You will not provide any explanations, and only return the Python Script without the explanation of the code. "
    },
    {
      "role": "user",
      "content": "\t\nWrite a function to find the longest chain which can be formed from the given set of pairs. The name of the function should be \n\nClass Pair(object)\n\ndef max_chain_length(arr, n)\n\n"
    }
  ],
  temperature=1,
  max_tokens=2047,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

In [29]:
print(response.choices[0].message.content)

```python
class Pair(object):
    def __init__(self, a, b):
        self.a = a
        self.b = b

def max_chain_length(arr, n):
    arr.sort(key = lambda x: x.b)

    max_length = 1
    prev = arr[0]
    for i in range(1, n):
        if arr[i].a > prev.b:
            max_length += 1
            prev = arr[i]
            
    return max_length
```


In [34]:
responseText = response.choices[0].message.content

# Remove the ```python and ``` from the response
responseText = responseText.replace("```python", "")
responseText = responseText.replace("```", "")

# Run the code
exec(responseText)

# Assertion Code:

assertion_code = [ "assert max_chain_length([Pair(5, 24), Pair(15, 25),Pair(27, 40), Pair(50, 60)], 4) == 3", "assert max_chain_length([Pair(1, 2), Pair(3, 4),Pair(5, 6), Pair(7, 8)], 4) == 4", "assert max_chain_length([Pair(19, 10), Pair(11, 12),Pair(13, 14), Pair(15, 16), Pair(31, 54)], 5) == 5" ]

# Execute and print if the code passes the assertions

for assertion in assertion_code:
    exec(assertion)

print("Code Passed Assertions")


Code Passed Assertions


In [65]:
def getCodeFormat(code_input):
    # Make an OPENAI API Call to only return the function call and its inputs
    '''EXAMPLE:
    INPUT: class Pair(object): def __init__(self, a, b): self.a = a self.b = b def max_chain_length(arr, n): max = 0 mcl = [1 for i in range(n)] for i in range(1, n): for j in range(0, i): if (arr[i].a > arr[j].b and mcl[i] < mcl[j] + 1): mcl[i] = mcl[j] + 1 for i in range(n): if (max < mcl[i]): max = mcl[i] return max
    
    OUTPUT: Class Pair(object) def max_chain_length(arr, n)
    '''

    client = OpenAI(
        api_key=os.environ.get('OPENAI_API_KEY',"")
    )

    response = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=[
            {
                "role": "system",
                "content": "Return only the function definition and its inputs from the code given by the user. Example: INPUT: class Pair(object): def __init__(self, a, b): self.a = a self.b = b def max_chain_length(arr, n): max = 0 mcl = [1 for i in range(n)] for i in range(1, n): for j in range(0, i): if (arr[i].a > arr[j].b and mcl[i] < mcl[j] + 1): mcl[i] = mcl[j] + 1 for i in range(n): if (max < mcl[i]): max = mcl[i] return max OUTPUT: Class Pair(object) def max_chain_length(arr, n) "
            },
            {
                "role": "user",
                "content": code_input
            }
        ],
        temperature=1,
        max_tokens=1024,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    
    responseText = response.choices[0].message.content

    # Remove the ```python and ``` from the response
    responseText = responseText.replace("```python", "")
    responseText = responseText.replace("```", "")

    return responseText

    

In [75]:
# The entire loop now for each examples

import jsonlines
import os
import openai
from openai import OpenAI

openai.api_key = os.environ.get('OPENAI_API_KEY',"")

max_loop = 10
i = 0

success = 0

# Open JSONL File:
with jsonlines.open('mbpp.jsonl') as reader: 
# Loop through each example
    for obj in reader:
        print("Running Example: " + str(i))
        prompt = obj['text']
        code = obj['code']
        #print(obj['code'])
        # Call the getCodeFormat function to get the function call and its inputs
        function_call = getCodeFormat(code)
        print(function_call)
        userPrompt =  prompt + " The name of the function should be "  + function_call + "\n\n"
        client = OpenAI(
  api_key=os.environ.get('OPENAI_API_KEY',""), 
)

        response = client.chat.completions.create(
        model="gpt-4-1106-preview",
        messages=[
            {
            "role": "system",
            "content": "You will generate a Python Script that will complete the task outlined by the user. You will not provide any explanations, and only return the Python Script without the explanation of the code. "
            },
            {
            "role": "user",
            "content": userPrompt
            }
        ],
        temperature=1,
        max_tokens=2047,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
        )

        responseText = response.choices[0].message.content

        # Remove the ```python and ``` from the response
        responseText = responseText.replace("```python", "")
        responseText = responseText.replace("```", "")
        
        # Run the code
        exec(responseText)

        # Assertion Code:
        assertion_code = obj['test_list']
        # Execute and print if the code passes the assertions
        for assertion in assertion_code:

            try:
                exec(assertion)
                print("Code Passed Assertions")
                success += 1
            except:
                # Continue to the next example and print the error
                print("Code Failed Assertions")
                break

        import time 
        time.sleep(5)
        
        # terminate loop for testing
        i += 1
        if i == max_loop:
            break




            
        
    


Running Example: 0
def count_common(words)
Code Failed Assertions
Running Example: 1
def find_Volume(l, b, h)
Code Passed Assertions
Code Passed Assertions
Code Passed Assertions
Running Example: 2
def split_lowerstring(text)
Code Failed Assertions
Running Example: 3
def text_lowercase_underscore(text)
Code Failed Assertions
Running Example: 4

def square_perimeter(a)

Code Passed Assertions
Code Passed Assertions
Code Passed Assertions
Running Example: 5
def str_to_list(string): 
def lst_to_string(List): 
def get_char_count_array(string): 
def remove_dirty_chars(string, second_string): 
Code Passed Assertions
Code Passed Assertions
Code Passed Assertions
Running Example: 6
def test_duplicate(arraynums)
Code Passed Assertions
Code Passed Assertions
Code Passed Assertions
Running Example: 7
def is_woodall(x)
Code Passed Assertions
Code Passed Assertions
Code Passed Assertions
Running Example: 8
def multiples_of_num(m, n)
Code Passed Assertions
Code Passed Assertions
Code Passed Assertio

In [74]:
print("Success: " + str(success) + " out of " + str(i))

Success: 16 out of 10


In [76]:
print("Success: " + str(success) + " out of " + str(i))

Success: 21 out of 10


In [77]:
# See number of examples in the JSONL file

with jsonlines.open('mbpp.jsonl') as reader:
    i = 0
    for obj in reader:
        i += 1
print("Number of Examples: " + str(i))

Number of Examples: 974


In [ ]:
!pip install heapq

In [7]:
import jsonlines
# Open label.jsonl file
with jsonlines.open('label.jsonl') as reader:
    # Count how many method2_gpt4_success
    # Count how many method2_gpt3_5_success

    i = 0
    j = 0

    for obj in reader:
        if obj['method2_gpt4_success'] == True:
            i += 1
        if obj['method2_gpt3_5_success'] == True:
            j += 1


print(i)
print(j)



564
517


In [9]:
# Open label.jsonl file
with jsonlines.open('label.jsonl') as reader:
    # Count how many method2_gpt4_success
    # Count how many method2_gpt3_5_success

    # find the number of the cases where gpt4 is success and gpt3.5 is not
    k = 0
    for obj in reader:
        if obj['method2_gpt4_success'] == True:
            if obj['method2_gpt3_5_success'] == False:
                # Print the problem:
                print(obj['text'])
                k+=1

print(k)


Write a function to find the number of ways to fill it with 2 x 1 dominoes for the given 3 x n board.
Write a function to convert the given binary number to its decimal equivalent.
Write a function to check whether all dictionaries in a list are empty or not.
Write a python function to find the sum of common divisors of two given numbers.
Write a function to calculate magic square.
Write a function to sort a list of elements using pancake sort.
Write a python function to find the sum of absolute differences in all pairs of the given array.
Write a python function to find k number of operations required to make all elements equal.
Write a function to calculate the nth pell number.
Write a function to count all the distinct pairs having a difference of k in any array.
Write a function to remove even characters in a string.
Write a python function to find the minimum element in a sorted and rotated array.
Write a function to find the number of possible sequences of length n such that each

In [6]:
# Open label.jsonl file
with jsonlines.open('label.jsonl') as reader:
    # Count how many method2_gpt4_success
    # Count how many method2_gpt3_5_success

    # find the number of the cases where gpt4 is success and gpt3.5 is not
    k = 0
    for obj in reader:
        if obj['method2_gpt4_success'] == False:
            if obj['method2_gpt3_5_success'] == True:
                # Print the problem:
                print(obj['text'])
                print(obj['method2_gpt4_output'])
                print(obj['method2_gpt3_5_output'])
                
                for i in obj["test_list"]:
                    exec(obj['method2_gpt4_output'])
                    exec(i)
                for i in obj["test_list"]:
                    exec(obj['method2_gpt3_5_output'])
                    exec(i)



                k+=1

print(k)


Write a function to find the number of ways to fill it with 2 x 1 dominoes for the given 3 x n board.

def count_ways(n):
    a, b = 1, 0
    for _ in range(n):
        a, b = b, a + b
    return b

# Testing the function with a value for n
# Uncomment the next line to use
# print(count_ways(3)) # Example usage


def count_ways(n):
    if n % 2 != 0:
        return 0
    dp = [0] * (n + 1)
    dp[0] = 1
    dp[2] = 3
    for i in range(4, n + 1, 2):
        dp[i] = dp[i - 2] * 3
        for j in range(4, i + 1, 2):
            dp[i] += dp[i - j] * 2
    return dp[n]



AssertionError: 